In [2]:
import numpy as np

import pandas as pd
from pandas.api.types import CategoricalDtype

%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn import linear_model as lm

import warnings
warnings.filterwarnings("ignore")

import zipfile
import os

#from ds100_utils import run_linear_regression_test

# Plot settings
plt.rcParams['figure.figsize'] = (12, 9)
plt.rcParams['font.size'] = 12

In [3]:
#training_data = maybe merge the Ahmed data with the insurance?

As a good sanity check, we should at least verify that the data shape matches the description.

In [4]:

# 204792 observations and 62 features in training data
#assert training_data.shape == (204792, 62)
# 68264 observations and 61 features in test data
#assert test_data.shape == (68264, 61)
# Sale Price is provided in the training data
#assert 'Sale Price' in training_data.columns.values
# Sale Price is hidden in the test data
#assert 'Sale Price' not in test_data.columns.values

We're now asking ourselves some questions about the data:

Who would be interested in seeing the effect of anxiety symptoms based on access to health care?

Who would be...[insert some bullshit]

### split training & test data

In [5]:
# This makes the train-test split in this section reproducible across different runs 
# of the notebook. You do not need this line to run train_test_split in general

# DO NOT CHANGE THIS LINE
np.random.seed(1337)
# DO NOT CHANGE THIS LINE

def train_test_split(data):
    data_len = data.shape[0]
    shuffled_indices = np.random.permutation(data_len)
    train_indices = shuffled_indices[:int(data_len//(1.25))]
    test_indices = shuffled_indices[int(data_len//(1.25)):]
    return data.iloc[train_indices,:], data.iloc[test_indices,:]
     
train, test = train_test_split(training_data)

NameError: name 'training_data' is not defined

### Model fitting

First, we define some feature functions:

In [8]:
def remove_outliers(data, variable, lower=-np.inf, upper=np.inf):
    """
    Input:
      data (data frame): the table to be filtered
      variable (string): the column with numerical outliers
      lower (numeric): observations with values lower than this will be removed
      upper (numeric): observations with values higher than this will be removed
    
    Output:
      a data frame with outliers removed
      
    Note: This function should not change mutate the contents of data.
    """  
    data = data[data[variable] > lower]
    data = data[data[variable] < upper]
    return data


def ohe_column(data, column):
    """
    One-hot-encodes roof material. New columns are of the form x0_MATERIAL.
    """
    oh_enc = OneHotEncoder()
    oh_enc.fit(data[[column]])
    dummies = pd.DataFrame(oh_enc.transform(data[[column]]).todense(), columns=oh_enc.get_feature_names_out(),index = data.index)

    return data.join(dummies)


In [6]:
from feature_func import *    # imports functions from Project 1A
# run this cell to define process_data_gm and select_columns

def process_data_gm(data, pipeline_functions, prediction_col):
    """Process the data for a guided model."""
    for function, arguments, keyword_arguments in pipeline_functions:
        if keyword_arguments and (not arguments):
            data = data.pipe(function, **keyword_arguments)
        elif (not keyword_arguments) and (arguments):
            data = data.pipe(function, *arguments)
        else:
            data = data.pipe(function)
    X = data.drop(columns=[prediction_col]).to_numpy()
    y = data.loc[:, prediction_col].to_numpy()
    return X, y

ModuleNotFoundError: No module named 'feature_func'